In [1]:
from dotenv import load_dotenv
import os
load_dotenv('./.env')
assert os.getenv('ENGINE') is not None, "Please set the ENGINE_STRING environment variable in the .env file"

In [3]:
from omop_constructs.alchemy.modifiers.modifier_joins import grade_select, t_stage_select, n_stage_select, m_stage_select, group_stage_select, size_select
from omop_constructs.semantics import registry, registry_engine
from omop_alchemy.cdm.model import Condition_Occurrence, Concept, Episode_Event
from omop_semantics.runtime.default_valuesets import runtime

print(t_stage_select.compile(registry_engine, compile_kwargs={"literal_binds": True}))

SELECT t_stage_ranked.person_id, t_stage_ranked.stage_id, t_stage_ranked.stage_date, t_stage_ranked.stage_datetime, t_stage_ranked.stage_concept_id, t_stage_ranked.measurement_event_id, t_stage_ranked.meas_event_field_concept_id, t_stage_ranked.stage_label, t_stage_ranked.stage_type, t_stage_ranked.rn 
FROM (SELECT t_stage_initial.person_id AS person_id, t_stage_initial.stage_id AS stage_id, t_stage_initial.stage_date AS stage_date, t_stage_initial.stage_datetime AS stage_datetime, t_stage_initial.stage_concept_id AS stage_concept_id, t_stage_initial.measurement_event_id AS measurement_event_id, t_stage_initial.meas_event_field_concept_id AS meas_event_field_concept_id, t_stage_initial.stage_label AS stage_label, t_stage_initial.stage_type AS stage_type, row_number() OVER (PARTITION BY t_stage_initial.measurement_event_id ORDER BY t_stage_initial.stage_type, t_stage_initial.stage_date ASC) AS rn 
FROM (SELECT measurement.person_id AS person_id, measurement.measurement_id AS stage_id, m

In [4]:
list(size_select.c)

[Column('person_id', Integer(), ForeignKey('person.person_id'), table=<tumor_size_earliest_filtered>, nullable=False),
 Column('measurement_event_id', Integer(), table=<tumor_size_earliest_filtered>),
 Column('meas_event_field_concept_id', Integer(), ForeignKey('concept.concept_id'), table=<tumor_size_earliest_filtered>),
 Column('measurement_concept_id', Integer(), ForeignKey('concept.concept_id'), table=<tumor_size_earliest_filtered>, nullable=False),
 Column('measurement_id', Integer(), table=<tumor_size_earliest_filtered>, primary_key=True, nullable=False),
 Column('measurement_date', Date(), table=<tumor_size_earliest_filtered>, nullable=False),
 Column('concept_name', String(length=255), table=<tumor_size_earliest_filtered>, nullable=False),
 Column('value_as_number', Float(), table=<tumor_size_earliest_filtered>),
 Column('unit_concept_id', Integer(), ForeignKey('concept.concept_id'), table=<tumor_size_earliest_filtered>),
 <sqlalchemy.sql.elements.ColumnClause at 0x12429db50; r

In [6]:
from omop_constructs.alchemy.modifiers import TStageMV, NStageMV, MStageMV, GroupStageMV, SizeModifierMV, GradeModifierMV

In [7]:
import sqlalchemy as sa
import sqlalchemy.orm as so

In [8]:
condition_concept = so.aliased(Concept, name='condition_concept')

In [9]:
TStageMV.__table__

Table('t_stage_mv', MetaData(), Column('person_id', Integer(), table=<t_stage_mv>), Column('stage_id', NullType(), table=<t_stage_mv>, primary_key=True, nullable=False), Column('stage_date', Date(), table=<t_stage_mv>), Column('stage_datetime', DateTime(), table=<t_stage_mv>), Column('stage_concept_id', Integer(), table=<t_stage_mv>), Column('stage_label', String(), table=<t_stage_mv>), Column('measurement_event_id', Integer(), table=<t_stage_mv>), Column('meas_event_field_concept_id', Integer(), table=<t_stage_mv>), schema=None)

In [10]:
print(TStageMV.__table__.compile(registry_engine, compile_kwargs={"literal_binds": True}))


In [12]:
from omop_constructs.alchemy.modifiers.condition_modifier_join import modified_conditions_join
from omop_constructs.alchemy.modifiers.condition_modifier_mv import ModifiedCondition

In [13]:
print(modified_conditions_join.compile(registry_engine, compile_kwargs={"literal_binds": True}))

SELECT row_number() OVER () AS mv_id, condition_occurrence.person_id, condition_occurrence.condition_start_date, condition_occurrence.condition_occurrence_id, condition_occurrence.condition_source_value, condition_occurrence.condition_concept_id, condition_concept.concept_name AS condition_concept, episode_event.episode_id AS condition_episode, t_stage_mv.stage_id AS t_stage_id, t_stage_mv.stage_date AS t_stage_date, t_stage_mv.stage_concept_id AS t_stage_concept_id, t_stage_mv.stage_label AS t_stage_label, n_stage_mv.stage_id AS n_stage_id, n_stage_mv.stage_date AS n_stage_date, n_stage_mv.stage_concept_id AS n_stage_concept_id, n_stage_mv.stage_label AS n_stage_label, m_stage_mv.stage_id AS m_stage_id, m_stage_mv.stage_date AS m_stage_date, m_stage_mv.stage_concept_id AS m_stage_concept_id, m_stage_mv.stage_label AS m_stage_label, group_stage_mv.stage_id AS group_stage_id, group_stage_mv.stage_date AS group_stage_date, group_stage_mv.stage_concept_id AS group_stage_concept_id, group_

In [14]:
registry['metastatic_disease'].lookup('Metastasis to heart')

35226167

In [15]:
registry['metastatic_disease'].lookup('Metastatic malignant neoplasm to epicardium')

4129922

In [16]:
runtime.condition_modifiers.condition_modifier_values.metastatic_disease

36769180

In [ ]:
with so.Session(registry_engine) as session:
    ModifiedCondition.refresh_mv(session)